In [69]:
# I/O
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import geopandas as gpd
import requests
from io import BytesIO
from zipfile import ZipFile
from datetime import datetime, timedelta

In [2]:
# URL of the zipped data file
urls = ['https://s3.amazonaws.com/capitalbikeshare-data/202401-capitalbikeshare-tripdata.zip',
        'https://s3.amazonaws.com/capitalbikeshare-data/202402-capitalbikeshare-tripdata.zip', 
       'https://s3.amazonaws.com/capitalbikeshare-data/202403-capitalbikeshare-tripdata.zip']

In [3]:
# List to store DataFrames
dataframes = []
# Loop through each URL
for url in urls:
    # Download the zipped file
    response = requests.get(url)
    zipfile = ZipFile(BytesIO(response.content))
    
    # List files in the zip file
    file_name = zipfile.namelist()[0]
    
    # Read the desired file into a pandas DataFrame
    with zipfile.open(file_name) as file:
        combined_df = pd.read_csv(file)
        dataframes.append(combined_df)

# Concatenate all DataFrames into one
combined_combined_df = pd.concat(dataframes, ignore_index=True)

In [4]:
# Display the DataFrame
combined_combined_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,748A93D7DE8A41CD,classic_bike,2024-01-25 15:49:59,2024-01-25 15:52:35,1st & O St NW,31519.0,1st & L St NW,31677.0,38.908643,-77.012365,38.903819,-77.011987,member
1,75CBFD136F06305B,classic_bike,2024-01-02 16:44:58,2024-01-02 16:53:25,1st & O St NW,31519.0,4th & College St NW,31138.0,38.908643,-77.012365,38.921233,-77.018135,member
2,0536C9720F87E04C,classic_bike,2024-01-24 15:40:15,2024-01-24 15:43:55,1st & O St NW,31519.0,1st & L St NW,31677.0,38.908643,-77.012365,38.903819,-77.011987,member
3,9E17390C218783B5,classic_bike,2024-01-04 15:35:00,2024-01-04 15:37:35,1st & O St NW,31519.0,1st & L St NW,31677.0,38.908643,-77.012365,38.903819,-77.011987,member
4,00727D0E773CDFF7,electric_bike,2024-01-05 12:27:58,2024-01-05 12:35:40,1st & O St NW,31519.0,10th & G St NW,31274.0,38.908690,-77.012317,38.898243,-77.026235,casual


# Exploratory Data Analysis  

Let's start creating new variables within our dataset and taking a look at its structure. 

In [5]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 436946 entries, 0 to 436945
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             436946 non-null  object 
 1   rideable_type       436946 non-null  object 
 2   started_at          436946 non-null  object 
 3   ended_at            436946 non-null  object 
 4   start_station_name  371894 non-null  object 
 5   start_station_id    371894 non-null  float64
 6   end_station_name    368605 non-null  object 
 7   end_station_id      368500 non-null  float64
 8   start_lat           436946 non-null  float64
 9   start_lng           436946 non-null  float64
 10  end_lat             436558 non-null  float64
 11  end_lng             436558 non-null  float64
 12  member_casual       436946 non-null  object 
dtypes: float64(6), object(7)
memory usage: 43.3+ MB


Start creating variables: 
- Start and End Dates
- Start and End Times
- Convert Station IDs to Character Integers
- Create new geocombined_dfs that have start and end locations so that we can create the neighborhoods that they started and ended in
- 

In [6]:
# Adding time_of_day
def get_time_of_day(hour):
    if 5 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 17:
        return 'afternoon'
    elif 17 <= hour < 21:
        return 'evening'
    else:
        return 'night'

federal_holidays_2024 = [
    '2024-01-01',  # New Year's Day
    '2024-01-15',  # Martin Luther King Jr. Day
    '2024-02-19',  # Presidents' Day
    '2024-05-27',  # Memorial Day
    '2024-06-19',  # Juneteenth National Independence Day
    '2024-07-04',  # Independence Day
    '2024-09-02',  # Labor Day
    '2024-10-14',  # Columbus Day
    '2024-11-11',  # Veterans Day
    '2024-11-28',  # Thanksgiving Day
    '2024-12-25'   # Christmas Day
]

# Adding holiday_weekend
def is_holiday_weekend(date):
    dt = pd.to_datetime(date)
    if dt.strftime('%Y-%m-%d') in federal_holidays_2024 and dt.weekday() in [1, 2, 3]:  # Tuesday, Wednesday, Thursday
        return True
    return False

In [7]:
combined_df["started_at"]= pd.to_datetime(combined_df["started_at"])
combined_df["started_at_date"]= pd.to_datetime(combined_df["started_at"]).dt.date
combined_df["started_at_time"]= pd.to_datetime(combined_df["started_at"]).dt.time
combined_df["ended_at"]= pd.to_datetime(combined_df["ended_at"])
combined_df["ended_at_date"]= pd.to_datetime(combined_df["ended_at"]).dt.date
combined_df["ended_at_time"]= pd.to_datetime(combined_df["ended_at"]).dt.time
combined_df["duration"] = pd.to_datetime(combined_df["ended_at"]) - pd.to_datetime(combined_df["started_at"])
combined_df["duration_minutes"] = combined_df.duration.apply(lambda td: td.total_seconds() / 60)
combined_df["start_station_id"] = combined_df["start_station_id"].astype(str)
combined_df["end_station_id"] = combined_df["end_station_id"].astype(str)
combined_df['time_of_day'] = combined_df['started_at'].apply(lambda x: get_time_of_day(pd.to_datetime(x).hour))
combined_df['day_of_week'] = combined_df['started_at'].apply(lambda x: pd.to_datetime(x).strftime('%A'))
# Adding day_type
combined_df['day_type'] = combined_df['day_of_week'].apply(lambda x: 'weekend' if x in ['Saturday', 'Sunday'] else 'work-day')
# Adding falls_on_holiday
combined_df['falls_on_holiday'] = combined_df['started_at'].apply(lambda x: pd.to_datetime(x).strftime('%Y-%m-%d') in federal_holidays_2024)
combined_df['holiday_weekend'] = combined_df['started_at'].apply(lambda x: is_holiday_weekend(x))
# Filter dataframe to only have rides between 30 seconds and 2 hours
og_df_size = combined_df.shape[0]
combined_df = combined_df.loc[(combined_df['duration_minutes']>=.5) & (combined_df['duration_minutes']<=120) ]

In [8]:
og_df_size
new_df_size = combined_df.shape[0]
new_df_size/og_df_size

0.9777546882223432

In [9]:
combined_df.describe()

,started_at,ended_at,start_lat,start_lng,end_lat,end_lng,duration,duration_minutes
count,427226,427226,427226.000000,427226.000000,427226.000000,427226.000000,427226,427226.000000
mean,2024-03-17 10:53:56.171096832,2024-03-17 11:08:53.587810816,38.904837,-77.031332,38.903965,-77.031151,0 days 00:14:57.416713870,14.956945
min,2024-03-01 00:00:22,2024-03-01 00:07:47,38.760000,-77.370000,38.710000,-77.440000,0 days 00:00:30,0.500000
25%,2024-03-10 15:43:06.249999872,2024-03-10 15:56:52,38.892223,-77.043074,38.890540,-77.043182,0 days 00:05:49,5.816667
50%,2024-03-17 12:01:29,2024-03-17 12:19:46,38.903750,-77.030000,38.903040,-77.030000,0 days 00:10:07,10.116667
75%,2024-03-24 18:41:54.249999872,2024-03-24 19:00:39.750000128,38.917668,-77.012344,38.915544,-77.012289,0 days 00:17:40,17.666667
max,2024-03-31 23:58:41,2024-04-01 00:50:05,39.125828,-76.820000,39.125828,-76.820000,0 days 02:00:00,120.000000
std,NaN,NaN,0.026159,0.034113,0.026088,0.034024,0 days 00:15:35.989089177,15.599818


Create two new dataframes: 
1. Start times combined_df that has the location they started at if it has a station
2. End times combined_df that has the location they ended at if it has a station 

In [10]:
start_stations = combined_df[["start_station_name","start_station_id","start_lat","start_lng"]]
end_stations = combined_df[["end_station_name","end_station_id","end_lat","end_lng"]]
start_stations = start_stations.loc[start_stations['start_station_name'].notna()]
end_stations = end_stations.loc[end_stations['end_station_name'].notna()]
start_stations.rename(columns = {"start_station_name":"station_name",
                                "start_station_id":"station_id",
                                "start_lat":"lat",
                                 "start_lng":"lng"}, inplace=True)

end_stations.rename(columns = {"end_station_name":"station_name",
                                "end_station_id":"station_id",
                                "end_lat":"lat",
                                 "end_lng":"lng"}, inplace=True )
stations = pd.concat([start_stations, end_stations])
stations = stations.drop_duplicates(subset=["station_name","station_id"])

In [11]:
combined_gdf = gpd.GeoDataFrame(stations, geometry=gpd.points_from_xy(stations.lng, stations.lat), crs="EPSG:4326")

In [12]:
combined_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 824 entries, 0 to 344888
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   station_name  824 non-null    object  
 1   station_id    824 non-null    object  
 2   lat           824 non-null    float64 
 3   lng           824 non-null    float64 
 4   geometry      824 non-null    geometry
dtypes: float64(2), geometry(1), object(2)
memory usage: 38.6+ KB


# Pull in Weather Data to add to rides

In [49]:
# Function to fetch weather data
def fetch_weather_data(lat, lon, date, api_key):
    url = "https://api.openweathermap.org/data/3.0/onecall/day_summary?"
    params = {
        'lat': lat,
        'lon': lon,
        'date': date.strftime('%Y-%m-%d'),
        'appid': api_key,
        'units':"imperial"
    }
    response = requests.get(url, params=params)
    return response.json()
# Parameters
api_key = "32964f206a33b141c84d7f13e55a92d4"
dc_lat = 38.8950368
dc_lon = -77.0365427
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 3, 31)

# Create a list of dates for each day within the date range
date_range = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]


In [50]:
# Fetch data
weather_data = fetch_weather_data(dc_lat, dc_lon, start_date, api_key)

In [51]:
weather_data

{'lat': 38.8950368,
 'lon': -77.0365427,
 'tz': '-05:00',
 'date': '2024-01-01',
 'units': 'imperial',
 'cloud_cover': {'afternoon': 100.0},
 'humidity': {'afternoon': 69.0},
 'precipitation': {'total': 1.92},
 'temperature': {'min': 37.89,
  'max': 41.83,
  'afternoon': 41.68,
  'night': 40.59,
  'evening': 38.08,
  'morning': 38.53},
 'pressure': {'afternoon': 1016.0},
 'wind': {'max': {'speed': 13.8, 'direction': 320.0}}}

In [52]:
# Fetch data
daily_data = []
for date in date_range:
    weather_data = fetch_weather_data(dc_lat, dc_lon, date, api_key)
    daily_data.append({
        'date': weather_data['date'],
        'min_temp': weather_data['temperature']['min'],
        'max_temp': weather_data['temperature']['max'],
        'afternoon_temp': weather_data['temperature']['afternoon'],
        'night_temp': weather_data['temperature']['night'],
        'evening_temp': weather_data['temperature']['evening'],
        'morning_temp': weather_data['temperature']['morning'],
        'humidity': weather_data['humidity']['afternoon'],
        'pressure': weather_data['pressure']['afternoon'],
        'wind_speed': weather_data['wind']['max']['speed'],
        'wind_direction': weather_data['wind']['max']['direction'],
        'precipitation': weather_data['precipitation']['total'],
        'cloud_cover': weather_data['cloud_cover']['afternoon']
    })

In [53]:
# Process and save data
weather_df = pd.DataFrame(daily_data)

In [54]:
# Ensure 'started_at' is a datetime
combined_df['started_at'] = pd.to_datetime(combined_df['started_at'])

# Create a 'date' column in datetime format
combined_df['date'] = combined_df['started_at'].dt.date
combined_df['date'] = pd.to_datetime(combined_df['date'])

# Melt the weather data to make times of the day as rows
weather_melted = weather_df.reset_index().melt(id_vars=['date'], 
                                               value_vars=['afternoon_temp', 'night_temp', 'evening_temp', 'morning_temp'], 
                                               var_name='time_of_day', 
                                               value_name='temperature')
# Ensure 'started_at' is a datetime
weather_melted['date'] = pd.to_datetime(weather_melted['date'])

# Map the time_of_day to match combined_df format
time_mapping = {
    'afternoon_temp': 'afternoon',
    'night_temp': 'night',
    'evening_temp': 'evening',
    'morning_temp': 'morning'
}
weather_melted['time_of_day'] = weather_melted['time_of_day'].map(time_mapping)

/tmp/ipykernel_2242/1258420122.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['started_at'] = pd.to_datetime(combined_df['started_at'])
/tmp/ipykernel_2242/1258420122.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['date'] = combined_df['started_at'].dt.date
/tmp/ipykernel_2242/1258420122.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [55]:
# Merge based on 'date' and 'time_of_day'
merged_df = pd.merge(combined_df, weather_melted, how='left', left_on=['date', 'time_of_day'], right_on=['date', 'time_of_day'])

# Start Analysis for Underserved Rides 

The idea for this analysis is that if a rider completes their ride outside of a dock, they either had to find a place nearby to park their bike because a nearby dock was full OR because there simply was not a dock nearby. According to the National Association of City Transportation Officials's [Bike Share Station Siting Guide](https://nacto.org/publication/bike-share-station-siting-guide/), docks should be stationed approximately 1,000 feet, or a 5-minute walk from one another. 

In [64]:
from sklearn.model_selection import train_test_split

In [65]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 427226 entries, 0 to 427225
Data columns (total 25 columns):
 #   Column              Non-Null Count   Dtype          
---  ------              --------------   -----          
 0   ride_id             427226 non-null  object         
 1   rideable_type       427226 non-null  object         
 2   started_at          427226 non-null  datetime64[ns] 
 3   ended_at            427226 non-null  datetime64[ns] 
 4   start_station_name  364340 non-null  object         
 5   start_station_id    427226 non-null  object         
 6   end_station_name    362220 non-null  object         
 7   end_station_id      427226 non-null  object         
 8   start_lat           427226 non-null  float64        
 9   start_lng           427226 non-null  float64        
 10  end_lat             427226 non-null  float64        
 11  end_lng             427226 non-null  float64        
 12  member_casual       427226 non-null  object         
 13  started_at_dat

In [ ]:
# Handle missing values
merged_df.fillna(method='ffill', inplace=True)

# Create target variable: 'ended_at_dock' (1 if ended at a dock, 0 if not)
merged_df['ended_at_dock'] = merged_df['end_station_id'].apply(lambda x: 1 if pd.notnull(x) else 0)

# Split the data into training and testing sets
X = merged_df.drop(['ended_at_dock'], axis=1)
y = merged_df['ended_at_dock']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [66]:
bikeshare_loc= gpd.read_file("data/Capital_Bikeshare_Locations.geojson")

/home/codespace/anaconda3/envs/final-project-env/lib/python3.12/site-packages/geopandas/io/file.py:399: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")
/home/codespace/anaconda3/envs/final-project-env/lib/python3.12/site-packages/geopandas/io/file.py:399: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")


In [72]:
merged_df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,started_at_date,started_at_time,ended_at_date,ended_at_time,duration,duration_minutes,time_of_day,day_of_week,day_type,falls_on_holiday,holiday_weekend,temperature
0,4E026D43FD09E59C,classic_bike,2024-03-11 17:46:29,2024-03-11 18:03:07,22nd & H St NW,31127.0,15th & W St NW,31125.0,38.898925,-77.048852,38.919018,-77.034449,member,2024-03-11,17:46:29,2024-03-11,18:03:07,0 days 00:16:38,16.633333,evening,Monday,work-day,False,False,55.42
1,AB210B914033D41B,classic_bike,2024-03-17 19:31:24,2024-03-17 19:39:34,Crystal Dr & 15th St S,31003.0,Pentagon City Metro / 12th St & S Hayes St,31005.0,38.861056,-77.049417,38.862303,-77.059936,member,2024-03-17,19:31:24,2024-03-17,19:39:34,0 days 00:08:10,8.166667,evening,Sunday,weekend,False,False,66.07
2,3B328C72BC05FDAB,classic_bike,2024-03-07 14:32:34,2024-03-07 15:19:45,Crystal Dr & 15th St S,31003.0,Crystal Dr & 15th St S,31003.0,38.861056,-77.049417,38.861056,-77.049417,casual,2024-03-07,14:32:34,2024-03-07,15:19:45,0 days 00:47:11,47.183333,afternoon,Thursday,work-day,False,False,61.99
3,A2FD150593E11106,classic_bike,2024-03-29 18:44:08,2024-03-29 18:49:59,Columbia Rd & Belmont St NW,31113.0,Massachusetts Ave & Dupont Circle NW,31200.0,38.920669,-77.043680,38.910100,-77.044400,member,2024-03-29,18:44:08,2024-03-29,18:49:59,0 days 00:05:51,5.850000,evening,Friday,work-day,False,False,59.54
4,4E18243CAADD3542,classic_bike,2024-03-24 11:18:00,2024-03-24 11:24:28,Columbia Rd & Belmont St NW,31113.0,Massachusetts Ave & Dupont Circle NW,31200.0,38.920669,-77.043680,38.910100,-77.044400,member,2024-03-24,11:18:00,2024-03-24,11:24:28,0 days 00:06:28,6.466667,morning,Sunday,weekend,False,False,31.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427221,2C8DA795B6EB641D,classic_bike,2024-03-18 11:28:11,2024-03-18 11:38:28,Maine Ave & Water St SW,31674.0,Jefferson Memorial,31249.0,38.876747,-77.020945,38.879819,-77.037413,casual,2024-03-18,11:28:11,2024-03-18,11:38:28,0 days 00:10:17,10.283333,morning,Monday,work-day,False,False,42.82
427222,6ACD1590D828E5E0,classic_bike,2024-03-24 08:02:48,2024-03-24 08:22:01,16th & R St NW,31282.0,Jefferson Memorial,31249.0,38.912652,-77.036278,38.879819,-77.037413,casual,2024-03-24,08:02:48,2024-03-24,08:22:01,0 days 00:19:13,19.216667,morning,Sunday,weekend,False,False,31.26
427223,8D5C2C1D5343F47D,electric_bike,2024-03-20 15:24:20,2024-03-20 15:38:36,17th & P St NW,31330.0,4th & Florida Ave NE,31532.0,38.909903,-77.038405,38.907316,-77.000698,member,2024-03-20,15:24:20,2024-03-20,15:38:36,0 days 00:14:16,14.266667,afternoon,Wednesday,work-day,False,False,61.07
427224,B3A5866552D56897,electric_bike,2024-03-20 16:18:56,2024-03-20 16:36:57,Convention Center / 7th & M St NW,31223.0,4th & Florida Ave NE,31532.0,38.905784,-77.022379,38.907316,-77.000698,casual,2024-03-20,16:18:56,2024-03-20,16:36:57,0 days 00:18:01,18.016667,afternoon,Wednesday,work-day,False,False,61.07


In [71]:
bikeshare_loc

,NAME,STATION_TYPE,STATION_ID,STATION_STATUS,LAST_REPORTED,NUM_DOCKS_AVAILABLE,NUM_DOCKS_DISABLED,NUM_BIKES_AVAILABLE,NUM_EBIKES_AVAILABLE,NUM_BIKES_DISABLED,IS_INSTALLED,IS_RETURNING,IS_RENTING,HAS_KIOSK,IOS,ANDROID,ELECTRIC_BIKE_SURCHARGE_WAIVER,EIGHTD_HAS_KEY_DISPENSER,CAPACITY,RENTAL_METHODS,REGION_ID,REGION_NAME,GIS_ID,LATITUDE,LONGITUDE,GIS_LAST_MOD_DTTM,OBJECTID,geometry
0,M St & New Jersey Ave SE,classic,082486f0-1f3f-11e7-bf6b-3863bb334450,None,2024-07-17 14:10:56+00:00,11,0,5,1,1,YES,YES,YES,YES,https://dc.lft.to/lastmile_qr_scan,https://dc.lft.to/lastmile_qr_scan,NO,NO,17,"KEY,CREDITCARD",42,DCA-CABI,None,38.876219,-77.004169,2024-07-17 14:12:27+00:00,196398166,POINT (-77.00417 38.87622)
1,23rd St S & Hayes St,classic,99644655-ef85-4e13-a331-a344d88ded32,None,2024-07-17 14:11:02+00:00,9,0,6,0,0,YES,YES,YES,YES,https://dc.lft.to/lastmile_qr_scan,https://dc.lft.to/lastmile_qr_scan,NO,NO,15,"KEY,CREDITCARD",41,DCA-CABI,None,38.853002,-77.059496,2024-07-17 14:12:27+00:00,196398167,POINT (-77.05949 38.85301)
2,Montgomery College / W Campus Dr & Mannakee St,classic,0825aa94-1f3f-11e7-bf6b-3863bb334450,None,2024-07-17 14:11:02+00:00,12,0,8,1,1,YES,YES,YES,YES,https://dc.lft.to/lastmile_qr_scan,https://dc.lft.to/lastmile_qr_scan,NO,NO,21,"KEY,CREDITCARD",43,DCA-CABI,None,39.095661,-77.159048,2024-07-17 14:12:27+00:00,196398168,POINT (-77.15906 39.09567)
3,Georgia Ave & Dahlia St NW,classic,f8b331c6-d208-4b17-b788-1a17d2520c40,None,2024-07-17 14:11:02+00:00,16,0,3,1,0,YES,YES,YES,YES,https://dc.lft.to/lastmile_qr_scan,https://dc.lft.to/lastmile_qr_scan,NO,NO,19,"KEY,CREDITCARD",42,DCA-CABI,None,38.976061,-77.026747,2024-07-17 14:12:27+00:00,196398169,POINT (-77.02675 38.97607)
4,New Hampshire Ave & Ward Pl NW,classic,082492e1-1f3f-11e7-bf6b-3863bb334450,None,2024-07-17 14:10:56+00:00,0,0,16,3,3,YES,YES,YES,YES,https://dc.lft.to/lastmile_qr_scan,https://dc.lft.to/lastmile_qr_scan,NO,NO,19,"KEY,CREDITCARD",42,DCA-CABI,None,38.905711,-77.047318,2024-07-17 14:12:27+00:00,196398170,POINT (-77.04732 38.90572)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
773,Market Square / King St & Royal St,classic,082539e0-1f3f-11e7-bf6b-3863bb334450,None,2024-07-17 14:11:16+00:00,6,0,9,1,0,YES,YES,YES,YES,https://dc.lft.to/lastmile_qr_scan,https://dc.lft.to/lastmile_qr_scan,NO,NO,15,"KEY,CREDITCARD",40,DCA-CABI,None,38.804718,-77.043363,2024-07-17 14:12:28+00:00,196398939,POINT (-77.04336 38.80472)
774,4th & College St NW,classic,60b9672e-7668-431b-801b-affaccb680bb,None,2024-07-17 14:11:16+00:00,10,0,5,1,0,YES,YES,YES,YES,https://dc.lft.to/lastmile_qr_scan,https://dc.lft.to/lastmile_qr_scan,NO,NO,15,"KEY,CREDITCARD",42,DCA-CABI,None,38.921233,-77.018135,2024-07-17 14:12:28+00:00,196398940,POINT (-77.01814 38.92124)
775,4th & M St SW,classic,08247896-1f3f-11e7-bf6b-3863bb334450,None,2024-07-17 14:11:16+00:00,10,0,17,1,0,YES,YES,YES,YES,https://dc.lft.to/lastmile_qr_scan,https://dc.lft.to/lastmile_qr_scan,NO,NO,27,"KEY,CREDITCARD",42,DCA-CABI,None,38.876823,-77.017774,2024-07-17 14:12:28+00:00,196398941,POINT (-77.01777 38.87683)
776,Mississippi Ave & 19th St SE / THEARC,classic,72b359a7-f70e-4b97-aaf6-2b26f09f19b1,None,2024-07-17 14:11:16+00:00,13,0,6,0,0,YES,YES,YES,YES,https://dc.lft.to/lastmile_qr_scan,https://dc.lft.to/lastmile_qr_scan,NO,NO,19,"KEY,CREDITCARD",42,DCA-CABI,None,38.843770,-76.975570,2024-07-17 14:12:28+00:00,196398942,POINT (-76.97557 38.84378)
